In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [4]:
from langsmith import traceable
from langchain_core.prompts import ChatPromptTemplate

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open('polly_facts.txt', 'r') as file:
        polly_facts = file.read()
    return {"question": question, "facts": polly_facts}

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = fake_db_retrieval_step | prompt | llm

In [5]:
question = "Do you like animals?"
chain.invoke(question)

AIMessage(content='Do I like animals? Polly likes animal crackers!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 95, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d', 'finish_reason': 'stop', 'logprobs': None}, id='run-35837d90-9ce1-48bc-85d7-9a240fc6e869-0', usage_metadata={'input_tokens': 95, 'output_tokens': 11, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Evaluation

In [6]:
from langsmith.schemas import Example, Run

def did_polly_respond(root_run: Run, example: Example) -> dict:
    print(root_run.outputs["output"])
    score = len(root_run.outputs["output"].pretty_repr()) > 0
    return {"key": "did_respond", "score": int(score)}

In [7]:
from langsmith import evaluate

evaluate(
    chain.invoke,
    data="polly-ground-truth-qa",
    evaluators=[did_polly_respond],
    experiment_prefix="gpt-4o-mini"
)

View the evaluation results for experiment: 'gpt-4o-mini-f8bbff99' at:
https://smith.langchain.com/o/6d88c1b4-3f97-4eea-85e9-cbb8cb922139/datasets/852ba7cb-3a4e-4abd-9970-fa3588ce1343/compare?selectedSessions=c5e7cb13-bc52-49ae-b836-1c5ebc894c15




0it [00:00, ?it/s]

content='You asked, "What do you like to eat?" I like animal crackers!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 102, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d', 'finish_reason': 'stop', 'logprobs': None} id='run-cb5d1173-c978-4945-95ab-687cd32cbf65-0' usage_metadata={'input_tokens': 102, 'output_tokens': 17, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='Does Polly like playing soccer? Yes, Polly likes playing soccer!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 101, 'total_tokens': 115, 'completion_

,inputs.input,outputs.output,error,reference.output,feedback.did_respond,execution_time,example_id,id,inputs.Input,reference.Output
0,What do you like to eat?,"content='You asked, ""What do you like to eat?""...",None,{'id': 'run-3df129fb-63aa-46df-a822-cede758775...,1,0.692087,7b31f87c-f3c1-4561-82e1-93a4c867322e,aeb199e5-6700-459c-a3d3-e7bf6dde027c,NaN,NaN
1,NaN,"content='Does Polly like playing soccer? Yes, ...",None,NaN,1,0.788096,0d5d2269-83b3-4847-a683-ebcdc794481b,ed14c54f-de3b-4eb6-8551-af49b10a1f43,Does Polly like playing soccer?,"Yes, Polly likes playing soccer."
2,NaN,content='Does Polly like playing basketball? Y...,None,NaN,1,0.861337,53145dd4-5ad9-44a7-8708-2d4f8c4fa361,612cc8ba-0be7-4ee5-bff2-8ba16e4c63ca,Does Polly like playing basketball?,"Yes, Polly likes playing basketball, but is no..."
3,NaN,"content='Does Polly like animal crackers? Yes,...",None,NaN,1,0.478111,59632967-f651-4fa8-a190-89d55fdab23e,25bd6d52-ce34-4f3f-b2b2-8bd0ce4c84e4,Does Polly like animal crackers?,"Yes, Polly likes animal crackers."
4,NaN,"content='You asked, ""What is Polly good at?"" P...",None,NaN,1,0.617434,73e484e6-978e-416d-9011-0018e7cc4212,f82aa121-3828-43d0-b0ce-074f28cbfcee,What is Polly good at?,Polly is awesome at flying.
5,NaN,"content='Does Polly like goldfish? No, Polly d...",None,NaN,1,0.798873,d697b1ae-5d5c-464d-af39-093b2fc0f4e3,df06f775-b4e7-4293-afe7-15d17c27f398,Does Polly like goldfish?,"No, Polly does not like goldfish."
